# Dataset, DataLoader
---

### 1. Dezero에서의 Dataset, DataLodaer
---
Daatset을 설정한 뒤에 DataLoder로 설정한 Dataset을 끌고 온다.

Dataset 은 샘플과 정답(label)을 저장하고, DataLoader 는 Dataset 을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)로 감싼다.

> PyTorch의 도메인 특화 라이브러리들은 (FashionMNIST와 같은) 미리 준비해둔(pre-loaded) 다양한 데이터셋을 제공합니다.    
> 데이터셋은 torch.utils.data.Dataset 의 하위 클래스로 개별 데이터를 특정하는 함수가 구현되어 있습니다.    
> 이러한 데이터셋은 모델을 만들어보고(prototype) 성능을 측정(benchmark)하는데 사용할 수 있습니다.    
> 1. 비디오 데이터셋 - https://pytorch.org/vision/stable/datasets.html  
> 2. 텍스트 데이터셋 - https://pytorch.org/text/stable/datasets.html  
> 3. 오디오 데이터셋 - https://pytorch.org/audio/stable/datasets.html  


Dataset은 왜 만들었을까?

데이터가 100만개라면, 거대한 데이터를 하나의 ndarray 인스턴스로 처리하면 모든 원소를 한꺼번에 메모리에 올려야 한다. 

당연히 메모리는 부담스러운 상황

그렇기 때문에 이러한 문제에 대응이 가능하도록 Dataset 클래스를 만들었다.


### 2. Dezero의 Dataset(복습)
---

In [ ]:
import numpy as np

class Dataset: # Dezero의 Dataset
    def __init__(self, train=True, transform=None, target_transform=None):
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        if self.transform is None:
            self.transform = lambda x: x
        if self.target_transform is None:
            self.target_transform = lambda x: x

        self.data = None # 입력 데이터 보관
        self.label = None # 레이블 보관
        self.prepare()

    def __getitem__(self, index): # 객체에 슬라이싱이 가능하게 하는 특수 메소드 
        assert np.isscalar(index) # 만약에 받은 요소가 스칼라이면 실행
        if self.label is None: # 레이블이 존재치 않는다면
            return self.transform(self.data[index]), None # 값에 인덱싱해서 반환
        else:
            return self.transform(self.data[index]), self.target_transform(self.label[index])

    def __len__(self): # 데이터셋 길이 반환
        return len(self.data)

    def prepare(self): # 유도 쿨래스에서 마저 정의해야 한다. 
        pass

class Spiral(Dataset): # 이렇게 데이터셋에 각종 데이터를 가져올 수 있게끔 설정해 뒀을것이다. 
    def prepare(self): # 이런식으로 인스턴스 변수인 data와 label데이터를 설정한다
        self.data, self.label = get_spiral(self.train)

pytorch또한 위와 비슷한 형상을 띄고 있을것이다. 

여기서 중요한 것은 데이터셋은 이미 다 정의가 되어있다는 것

파이토치가 정해준 데이터셋을 그냥 이용하면 끝이다. 물론 튜닝을 할 수 있지만... 굳이?

### 3. pytorch의 dataset
---

기본적으로 제공되는 틀인 Dataset이라는 클래스가 존재한다.

이 클래스를 상속받고 

__init__(self) : 필요한 변수들을 선언하는 메서드. input으로 오는 x와 y를 load 하거나, 파일목록을 load한다.

__len__(self) : x나 y 는 길이를 넘겨주는 메서드.

__getitem__(self, index) : index번째 데이터를 return 하는 메서드.

들을 정의해주면 사용자 전용 Dataset 완성

찾아보니 __len__ 메서드와 __getitiem__ 메서드를 지닌 어떤 것도 죄다 Dataset이 될 수 있다고 한다. (물론 오류에 대한 책임은 작성자가 지어야 한다.)

> 사용자 정의 Dataset 클래스는 반드시 3개 함수를 구현해야 합니다: __init__, __len__, and __getitem__. 



In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self, index):
        pass
    def __len__(self):
        pass

### 4. Dezero의 Dataloader
---


In [3]:
import math
import random
import numpy as np

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True, gpu=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.data_size = len(dataset)
        self.max_iter = math.ceil(self.data_size / batch_size)
        self.gpu = gpu

        self.reset()

    def reset(self):
        self.iteration = 0 # 반복자를 0으로 설정, 반복자는 데이터를 순차적으로 추출하는 기능을 제공한다.
        if self.shuffle: 
            self.index = np.random.permutation(len(self.dataset))
        else:
            self.index = np.arange(len(self.dataset))

    def __iter__(self): # 클래스를 파이썬 반복자로 사용하기 위해 넣은 특수 메서드 
        return self # 특수 메서드를 구현하고 자기 자신을 반환하도록 하는게 기본 문법.

    def __next__(self): # 반복자에서 데이터를 순서대로 추출하기 위한 메서드
        if self.iteration >= self.max_iter:
            self.reset()
            raise StopIteration() # 다음 원소를 반환하도록 구현, 반환할 원소가 없으면 StopIteration()을 수행 

        i, batch_size = self.iteration, self.batch_size
        batch_index = self.index[i * batch_size:(i + 1) * batch_size]
        batch = [self.dataset[i] for i in batch_index]

        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        t = xp.array([example[1] for example in batch])

        self.iteration += 1
        return x, t # 데이터와 레이블을 반환

    def next(self):
        return self.__next__()

Dataset과는 달리 DataLoader는 그닥 사용자가 수정할 이유가 없다.

DataLoader는 그저 Dataset를 기반으로 데이터를 꺼내는 것일뿐 그 이상의 기능을 할 이유가 없다.

### 5. pytorch의 DataLoader
---
정의는 다음과 같다.

DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)


parameter 설명

1. dataset는 필수 입력 인자다.

2. bath_size = 1 : 배치 크기이다.

3. shuffle = False : 에포크별로 데이터셋을 뒤섞을건지 여부를 결정하는 것

나머지는 많이 사용하지는 않는다. 그 외 설명은 다음 글을 참고하길 바란다.
> DataLoader parameter 별 용도 : https://subinium.github.io/pytorch-dataloader/

In [ ]:
# DataLoader 사용 예제

from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

for epoch in range(max_epoch):
    for x, t in train_dataloader:
        print(x.shape, t.shape)

        ...
        

NameError: name 'training_data' is not defined